In [1]:
import os
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from keras.regularizers import l1_l2
from typhon.retrieval.qrnn import QRNN
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_excel(r'E:\华为电脑备份\研究生的森活\居超QRCDCNN论文\数据\日数据\数据合并过程.xlsx', sheet_name='损失率+滞后',index_col='DATE')
SP500=df.iloc[:,0]
BNY_MELLON=df.iloc[:,1]
BANK_OF_AMERICA=df.iloc[:,2]
CITIGROUP=df.iloc[:,3]
COMERICA=df.iloc[:,4]
HUNTINGTON=df.iloc[:,5]
JPMORGAN_CHASE=df.iloc[:,6]
MT_BANK=df.iloc[:,7]
PNC_Financial_Services=df.iloc[:,8]
STATE_STREET=df.iloc[:,9]
SYNOVUS_FINANCIAL=df.iloc[:,10]
WELLS_FARGO=df.iloc[:,11]
ZIONS_BANCORP=df.iloc[:,12]
GOLDMAN_SACHS=df.iloc[:,13]
MORGAN_STANLEY=df.iloc[:,14]
CHARLES_SCHWAB=df.iloc[:,15]
T_ROWE_PRICE=df.iloc[:,16]
AFLAC=df.iloc[:,17]
ALLSTATE=df.iloc[:,18]
AMERICAN_INTERNATIONAL=df.iloc[:,19]
AON=df.iloc[:,20]
BERKLEY_W_R=df.iloc[:,21]
CIGNA=df.iloc[:,22]
CAN_FINANCIAL=df.iloc[:,23]
CHUBB=df.iloc[:,24]
CINCINNATI=df.iloc[:,25]
HARTFORD_FINANCIAL=df.iloc[:,26]
HUMANA=df.iloc[:,27]
LINCOLN_NATIONAL=df.iloc[:,28]
LOEWS=df.iloc[:,29]
MARSH_MCLENNAN=df.iloc[:,30]
MBIA=df.iloc[:,31]
THE_PROGRESSIVE=df.iloc[:,32]
UNUM=df.iloc[:,33]
AMERICAN_EXPRESS=df.iloc[:,34]
FIFTH_THIRD=df.iloc[:,35]
FRANKLIN_RESOURCES=df.iloc[:,36]
SEI_INVESTMENTS=df.iloc[:,37]
UNION_PACIFIC=df.iloc[:,38]
three_month_yield_change=df.iloc[:,39]
term_spread_change=df.iloc[:,40]
TED_spread=df.iloc[:,41]
credit_spread_change=df.iloc[:,42]
market_return=df.iloc[:,43]
real_estate_excess_return=df.iloc[:,44]
equity_volatility=df.iloc[:,45]
state_variable=df.iloc[:,39:]

In [3]:
Dict1=[0,0,0,0]
# Dict2_VaR_QRNN_95=pd.DataFrame()
# Dict2_CoVaR_QRNN_95=pd.DataFrame()
# Dict2_VaR_QRNN_test_95=pd.DataFrame()
# Dict2_CoVaR_QRNN_test_95=pd.DataFrame()

VaR1_QRNN_95=pd.DataFrame()
VaR1_QRNN_99=pd.DataFrame()
VaR1_QRNN_test_95=pd.DataFrame()
VaR1_QRNN_test_99=pd.DataFrame()
CoVaR1_QRNN_95=pd.DataFrame()
CoVaR1_QRNN_99=pd.DataFrame()
CoVaR1_QRNN_test_95=pd.DataFrame()
CoVaR1_QRNN_test_99=pd.DataFrame()

for j in list(range(1,39)):
    x = state_variable.to_numpy()
    y=df.iloc[:,j].to_numpy()
    # 首先将数据拆分为训练集和临时集（包含验证集和测试集）
    x_temp, x_test, y_temp, y_test = train_test_split(x, y, test_size=0.05)
#     x_train_95, x_val_95, y_train_95, y_val_95 = train_test_split(x_temp, y_temp, test_size=0.5, random_state=j)

    # x=x.astype(np.float32)
    # y=y.astype(np.float32)
    input_dim = x.shape[1]
    quantiles = [0.95]

    model = QRNN(input_dimensions=input_dim, quantiles=quantiles, kernel_regularizer=l1_l2(l1=0.01, l2=0.01))
    history = model.train(training_data=(x_temp,y_temp),training_split=5/95,batch_size=64,maximum_epochs=100)

    a = model.predict(x)
    a_test = model.predict(x_test)
    
    VaR_QRNN_test_95 = pd.DataFrame(a_test[:,0])
    VaR1_QRNN_test_95=pd.concat([VaR1_QRNN_test_95,VaR_QRNN_test_95],axis=1)
    sxc_VaR_QRNN_test_95 = 0
    for i in range(len(y_test)):
        if y_test[i] > VaR_QRNN_test_95[0][i]:
            sxc_VaR_QRNN_test_95+=1
#     Dict1_VaR_QRNN_test_95=pd.DataFrame([sxc_VaR_QRNN_test_95],columns=['VaR_QRNN_test_95_失败天数'])
#     Dict2_VaR_QRNN_test_95=pd.concat([ Dict2_VaR_QRNN_test_95, Dict1_VaR_QRNN_test_95],axis=0)


    # b=a.reshape(a.shape[0],1)
    # b.shape
    VaR_QRNN_95 = pd.DataFrame(a[:,0])
    VaR1_QRNN_95=pd.concat([VaR1_QRNN_95,VaR_QRNN_95],axis=1)
    sxc_VaR_QRNN_95 = 0
    for i in range(len(y)):
        if y[i] > VaR_QRNN_95[0][i]:
            sxc_VaR_QRNN_95+=1
    print(sxc_VaR_QRNN_95)

#     VaR_QRNN_99 = pd.DataFrame(a[:,1])
#     VaR1_QRNN_99=pd.concat([VaR1_QRNN_99,VaR_QRNN_99],axis=1)
#     sxc_VaR_QRNN_99 = 0
#     for i in range(len(y)):
#         if y[i] > VaR_QRNN_99[0][i]:
#             sxc_VaR_QRNN_99+=1
#     print(sxc_VaR_QRNN_99)



    institution = df.iloc[:,j]
    concat = pd.concat([state_variable,institution],axis=1)
     # 把两个数据框合并起来，合并方式为按列合并
    x2=concat.to_numpy()[:,0:]
    y2=SP500.to_numpy()
    x2_temp, x2_test, y2_temp, y2_test = train_test_split(x2, y2, test_size=0.05)

    input_dim2 = x2.shape[1]
    print(input_dim)
    quantiles = [0.95]

    model2 = QRNN(input_dimensions=input_dim2, quantiles=quantiles)
    history2 = model2.train(training_data=(x2_temp,y2_temp),training_split=5/95,batch_size=64,maximum_epochs=100)

    a2_test = model2.predict(x2_test)
    CoVaR_QRNN_test_95 = pd.DataFrame(a2_test[:,0])
    CoVaR1_QRNN_test_95=pd.concat([CoVaR1_QRNN_test_95,CoVaR_QRNN_test_95],axis=1)
    sxc_CoVaR_QRNN_test_95 = 0
    for i in range(len(y2_test)):
        if  y2_test[i] > CoVaR_QRNN_test_95[0][i]:
            sxc_CoVaR_QRNN_test_95+=1
#     Dict1_CoVaR_QRNN_test_95=pd.DataFrame([sxc_CoVaR_QRNN_test_95],columns=['CoVaR_QRNN_test_95_失败天数'])
#     Dict2_CoVaR_QRNN_test_95=pd.concat([ Dict2_CoVaR_QRNN_test_95, Dict1_CoVaR_QRNN_test_95],axis=0)

    
    a2 = model2.predict(x2)
    print('a2.shape',np.shape(a2))
    CoVaR_QRNN_95 = pd.DataFrame(a2[:,0])
    CoVaR1_QRNN_95=pd.concat([CoVaR1_QRNN_95,CoVaR_QRNN_95],axis=1)
    sxc_CoVaR_QRNN_95 = 0
    for i in range(len(y2)):
        if  y2[i] > CoVaR_QRNN_95[0][i]:
            sxc_CoVaR_QRNN_95+=1
    print('sxc_CoVaR_QRNN_95:',sxc_CoVaR_QRNN_95)


#     CoVaR_QRNN_99 = pd.DataFrame(a[:,1])
#     CoVaR1_QRNN_99=pd.concat([CoVaR1_QRNN_99,CoVaR_QRNN_99],axis=1)
#     sxc_CoVaR_QRNN_99 = 0
#     for i in range(len(y)):
#         if y[i] > CoVaR_QRNN_99[0][i]:
#             sxc_CoVaR_QRNN_99+=1
#     print('sxc_CoVaR_QRNN_99:',sxc_CoVaR_QRNN_99)


    Dict =[sxc_VaR_QRNN_test_95,sxc_VaR_QRNN_95,sxc_CoVaR_QRNN_test_95,sxc_CoVaR_QRNN_95]
    print(Dict)
    Dict1= np.vstack((Dict1,Dict))
    print(Dict1)
    Dict2=pd.DataFrame(Dict1,columns=['VaR_QRNN_test_95','VaR_QRNN_95','sxc_CoVaR_QRNN_test_95','CoVaR_QRNN_95'])

Epoch 1/100


C:\Users\14867\AppData\Roaming\Python\Python310\site-packages\keras\optimizers\legacy\gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
C:\Users\14867\AppData\Roaming\Python\Python310\site-packages\typhon\retrieval\qrnn\models\keras.py:448: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  self.fit_generator(


36/36 [==============================] - 0s 812us/step - loss: 0.0025
Epoch 2/100
36/36 [==============================] - 0s 751us/step - loss: 0.0022
Epoch 3/100
36/36 [==============================] - 0s 756us/step - loss: 0.0022
Epoch 4/100
36/36 [==============================] - 0s 714us/step - loss: 0.0022
Epoch 5/100
36/36 [==============================] - 0s 718us/step - loss: 0.0022
Epoch 6/100
36/36 [==============================] - 0s 650us/step - loss: 0.0022
Epoch 7/100
36/36 [==============================] - 0s 750us/step - loss: 0.0022
Epoch 8/100
36/36 [==============================] - 0s 650us/step - loss: 0.0022
Epoch 9/100
36/36 [==============================] - 0s 757us/step - loss: 0.0022
Epoch 10/100
36/36 [==============================] - 0s 678us/step - loss: 0.0022
Epoch 11/100
36/36 [==============================] - 0s 735us/step - loss: 0.0022
Epoch 12/100
36/36 [==============================] - 0s 694us/step - loss: 0.0022
Epoch 13/100
36/36 [=====

4/4 [==============================] - 0s 907us/step
125
7
Epoch 1/100


C:\Users\14867\AppData\Roaming\Python\Python310\site-packages\keras\optimizers\legacy\gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
C:\Users\14867\AppData\Roaming\Python\Python310\site-packages\typhon\retrieval\qrnn\models\keras.py:448: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  self.fit_generator(


...

C:\Users\14867\AppData\Roaming\Python\Python310\site-packages\keras\optimizers\legacy\gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
C:\Users\14867\AppData\Roaming\Python\Python310\site-packages\typhon\retrieval\qrnn\models\keras.py:448: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  self.fit_generator(


36/36 [==============================] - 0s 2ms/step - loss: 0.0022
Epoch 2/100
36/36 [==============================] - 0s 2ms/step - loss: 0.0020
Epoch 3/100
36/36 [==============================] - 0s 2ms/step - loss: 0.0020
Epoch 4/100
36/36 [==============================] - 0s 2ms/step - loss: 0.0020
Epoch 5/100
36/36 [==============================] - 0s 2ms/step - loss: 0.0019
Epoch 6/100
36/36 [==============================] - 0s 2ms/step - loss: 0.0019
Epoch 7/100
36/36 [==============================] - 0s 2ms/step - loss: 0.0019
Epoch 8/100
36/36 [==============================] - 0s 2ms/step - loss: 0.0019
Epoch 9/100
36/36 [==============================] - 0s 1ms/step - loss: 0.0019
Epoch 10/100
36/36 [==============================] - 0s 2ms/step - loss: 0.0019
Epoch 11/100
36/36 [==============================] - 0s 2ms/step - loss: 0.0019
Epoch 12/100
36/36 [==============================] - 0s 2ms/step - loss: 0.0019
Epoch 13/100
36/36 [=============================

C:\Users\14867\AppData\Roaming\Python\Python310\site-packages\keras\optimizers\legacy\gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
C:\Users\14867\AppData\Roaming\Python\Python310\site-packages\typhon\retrieval\qrnn\models\keras.py:448: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  self.fit_generator(


36/36 [==============================] - 0s 2ms/step - loss: 0.0015
Epoch 2/100
36/36 [==============================] - 0s 2ms/step - loss: 0.0013
Epoch 3/100
36/36 [==============================] - 0s 2ms/step - loss: 0.0012
Epoch 4/100
36/36 [==============================] - 0s 2ms/step - loss: 0.0011
Epoch 5/100
36/36 [==============================] - 0s 2ms/step - loss: 0.0010
Epoch 6/100
36/36 [==============================] - 0s 2ms/step - loss: 9.8204e-04
Epoch 7/100
36/36 [==============================] - 0s 2ms/step - loss: 9.5446e-04
Epoch 8/100
36/36 [==============================] - 0s 1ms/step - loss: 9.3316e-04
Epoch 9/100
36/36 [==============================] - 0s 2ms/step - loss: 9.1825e-04
Epoch 10/100
36/36 [==============================] - 0s 2ms/step - loss: 8.6067e-04
Epoch 11/100
36/36 [==============================] - 0s 2ms/step - loss: 8.9581e-04
Epoch 12/100
36/36 [==============================] - 0s 2ms/step - loss: 8.8178e-04
Epoch 13/100
36/36 [=

36/36 [==============================] - 0s 1ms/step - loss: 7.5954e-04
Epoch 98/100
36/36 [==============================] - 0s 2ms/step - loss: 7.5290e-04
Epoch 99/100
36/36 [==============================] - 0s 2ms/step - loss: 7.5517e-04
Epoch 100/100
77/77 [==============================] - 0s 780us/step
a2.shape (2448, 1)
sxc_CoVaR_QRNN_95: 126
[6, 123, 9, 126]
[[  0   0   0   0]
 [  8 125   6 122]
 [  4 121   7 124]
 [  6 124   9 126]
 [ 12 129   6 123]
 [  7 125   6 123]
 [  9 125   4 121]
 [  9 126   7 124]
 [  8 125   4 119]
 [  4 121  15 132]
 [ 10 128   6 125]
 [  5 122   6 123]
 [  6 123  10 126]
 [  9 125   7 121]
 [  6 122   4 122]
 [  5 122   3 121]
 [  6 123   5 108]
 [  8 125   9 125]
 [  6 122   8 119]
 [  8 124   6 124]
 [  5 121   9 126]
 [  9 126   4 121]
 [  8 126   7 123]
 [  7 124   7 126]
 [  4 121  10 129]
 [  4 121   4 122]
 [ 11 128   5 122]
 [  9 126   5 122]
 [  9 126  10 127]
 [  9 126   7 133]
 [  8 125   7 124]
 [ 10 127  12 129]
 [  6 123   7 126]
 [ 

In [7]:
result=Dict2.drop(index=[0])
print(result)
result.to_excel("QRNN_result.xlsx", sheet_name="QRNN_result", index=False)

    VaR_QRNN_test_95  VaR_QRNN_95  sxc_CoVaR_QRNN_test_95  CoVaR_QRNN_95
1                  8          125                       6            122
2                  4          121                       7            124
3                  6          124                       9            126
4                 12          129                       6            123
5                  7          125                       6            123
6                  9          125                       4            121
7                  9          126                       7            124
8                  8          125                       4            119
9                  4          121                      15            132
10                10          128                       6            125
11                 5          122                       6            123
12                 6          123                      10            126
13                 9          125                  

In [8]:
result.mean()

VaR_QRNN_test_95            7.263158
VaR_QRNN_95               124.236842
sxc_CoVaR_QRNN_test_95      7.342105
CoVaR_QRNN_95             124.263158
dtype: float64

In [6]:
with pd.ExcelWriter('VaR_CoVaR_QRNN.xlsx') as writer:
    VaR1_QRNN_95.to_excel(writer,sheet_name="VaR1_QRNN_95", index=False)
    VaR1_QRNN_99.to_excel(writer,sheet_name='VaR1_QRNN_99', index=False)
    CoVaR1_QRNN_95.to_excel(writer,sheet_name='CoVaR1_QRNN_95', index=False)
    CoVaR1_QRNN_99.to_excel(writer,sheet_name='CoVaR1_QRNN_99', index=False)